# Python functies voor Odata3
Functies voor ophalen, inspecteren en samenvoegen van data van CBS
Verschillen met indeling data CBS op OData4:

file:///C:/Users/DKuipers/AppData/Local/Temp/odata4-overstappen-odata3.pdf

handleiding voor data CBS op Odata3:
file:///C:/Users/DKuipers/AppData/Local/Temp/cbs-open-data-services.pdf

Het voornaamste verschil is dat Odata3 al op de klassieke (tidy) manier observaties opslaat. Verder verschillende de namen van de tabellen in één set, en de namen van de metingen. Er zijn verschillen in welke data wordt gemeten, Odata 4 heeft bijvoorbeeld niet de kolom "meest voorkomende postcode".

### Nog te doen: zorg ervoor dat de filters daadwerkelijk filteren

In [49]:
import re
import numpy as np
import pandas as pd
import requests
import time

def get_odata(target_url):
    
    """"
    De functie gebruikt een API genaamd OData om data van het CBS op te halen.
    De data wordt in stukken opgehaald en in een pandas dataframe gezet.
    De URL moet er zo uitzien: "https://opendata.cbs.nl/ODataApi/odata/84583NED"
    De code van de tabel die je zoekt vindt je via Statline.
    Ga naar de data in Statline op de website van het CBS en kijk naar de URL om de code te vinden.
    """
    
    data = pd.DataFrame()
    while target_url:
        r = requests.get(target_url).json()
        data = data.append(pd.DataFrame(r['value']))
        
        if '@odata.nextLink' in r:
            target_url = r['@odata.nextLink']
        else:
            target_url = None
            
    return data

In [50]:
lastyear_url = "https://opendata.cbs.nl/ODataApi/odata/84583NED"

In [51]:
test = get_odata(lastyear_url)
test

,name,url
0,TableInfos,https://opendata.cbs.nl/ODataApi/odata/84583NE...
1,UntypedDataSet,https://opendata.cbs.nl/ODataApi/odata/84583NE...
2,TypedDataSet,https://opendata.cbs.nl/ODataApi/odata/84583NE...
3,DataProperties,https://opendata.cbs.nl/ODataApi/odata/84583NE...
4,CategoryGroups,https://opendata.cbs.nl/ODataApi/odata/84583NE...
5,WijkenEnBuurten,https://opendata.cbs.nl/ODataApi/odata/84583NE...


UntypedDataSet bevat ook symbolen zoals %.
TypedDataSet bevat de data in een vorm die bedoeld is om met de computer te verwerken. 

In [52]:
metadata = True
if metadata:
    test_info = pd.read_csv("ODATA3features.csv",sep=";", header=1, skiprows = None)
    print(test_info.columns)
    feature_selection = test_info[test_info['Type'] == "Topic"] 
else:
    test_info = get_odata(lastyear_url+"/TableInfos")
    print(test_info.columns)
    print(test_info[['ExplanatoryText','Frequency','ShortDescription','ShortTitle','Summary','Title']])
    # aan te raden om op te slaan als csv en te openen met kladblok.
    # Ook te verkrijgen door metadata te downloaden in Statline-portaal

Index(['ID', 'Position', 'ParentID', 'Type', 'Key', 'Title', 'Description',
       'ReleasePolicy', 'Datatype', 'Unit', 'Decimals', 'Default'],
      dtype='object')


In [53]:
feature_selection

,ID,Position,ParentID,Type,Key,Title,Description,ReleasePolicy,Datatype,Unit,Decimals,Default
2,2,1.0,1.0,Topic,Gemeentenaam_1,Gemeentenaam,De naam van de bestuurlijke gemeente. Deze naa...,NaN,String,naam,0.0,Impossible
3,3,2.0,1.0,Topic,SoortRegio_2,Soort regio,"De gekozen regioaanduiding betreft: Gemeente, ...",NaN,String,omschrijving,0.0,Impossible
4,4,3.0,1.0,Topic,Codering_3,Codering,"Gemeentecode heeft 4 posities, voorafgegaan do...",NaN,String,code,0.0,Impossible
5,5,4.0,1.0,Topic,IndelingswijzigingWijkenEnBuurten_4,Indelingswijziging wijken en buurten,Deze indicator geeft per wijk en buurt aan of ...,NaN,String,code,0.0,Impossible
7,7,5.0,6.0,Topic,AantalInwoners_5,Aantal inwoners,NaN,NaN,Long,aantal,0.0,Impossible
9,9,6.0,8.0,Topic,Mannen_6,Mannen,NaN,NaN,Long,aantal,0.0,Impossible
10,10,7.0,8.0,Topic,Vrouwen_7,Vrouwen,NaN,NaN,Long,aantal,0.0,Impossible
12,12,8.0,11.0,Topic,k_0Tot15Jaar_8,0 tot 15 jaar,Aantal inwoners dat op 1 januari 0 tot 15 jaar...,NaN,Long,aantal,0.0,Impossible
13,13,9.0,11.0,Topic,k_15Tot25Jaar_9,15 tot 25 jaar,Aantal inwoners dat op 1 januari 15 tot 25 jaa...,NaN,Long,aantal,0.0,Impossible
14,14,10.0,11.0,Topic,k_25Tot45Jaar_10,25 tot 45 jaar,Aantal inwoners dat op 1 januari 25 tot 45 jaa...,NaN,Long,aantal,0.0,Impossible


In [54]:
feature_selection['Key'][4]

'Codering_3'

In [55]:
test_wijken=get_odata(lastyear_url+"/WijkenEnBuurten")
test_wijken.head()
test_wijken.to_csv('Data/PlaatsenIndex.csv',sep=';')

In [56]:
test_props = get_odata(lastyear_url+"/DataProperties")
#print(test_props.columns)
print(test_props['Title'].values)
test_props.to_csv("DataPropertiesOData3.csv",sep=";")

['Wijken en buurten' 'Regioaanduiding' 'Gemeentenaam' 'Soort regio'
 'Codering' 'Indelingswijziging wijken en buurten' 'Bevolking'
 'Aantal inwoners' 'Geslacht' 'Mannen' 'Vrouwen' 'Leeftijdsgroepen'
 '0 tot 15 jaar' '15 tot 25 jaar' '25 tot 45 jaar' '45 tot 65 jaar'
 '65 jaar of ouder' 'Burgerlijke staat' 'Ongehuwd' 'Gehuwd' 'Gescheiden'
 'Verweduwd' 'Personen met een migratieachtergrond' 'Westers totaal'
 'Niet-westers' 'Niet-westers totaal' 'Marokko'
 'Nederlandse Antillen en Aruba' 'Suriname' 'Turkije'
 'Overig niet-westers' 'Geboorte en sterfte' 'Geboorte totaal'
 'Geboorte relatief' 'Sterfte totaal' 'Sterfte relatief'
 'Particuliere huishoudens' 'Huishoudens totaal' 'Eenpersoonshuishoudens'
 'Huishoudens zonder kinderen' 'Huishoudens met kinderen'
 'Gemiddelde huishoudensgrootte' 'Bevolkingsdichtheid' 'Wonen'
 'Woningvoorraad' 'Gemiddelde woningwaarde' 'Woningen naar type'
 'Percentage eengezinswoning' 'Percentage meergezinswoning'
 'Woningen naar bewoning' 'Percentage bewoond' 'P

In [57]:
#test_props.head(20)

In [58]:
test_observations = get_odata(lastyear_url+"/TypedDataSet"+"?$top=10")

In [59]:
#print(list(test_observations.columns))
print(test_observations["Gemeentenaam_1"])
print(test_observations["SoortRegio_2"])
print(test_observations["Codering_3"])
#print(test_observations["MeestVoorkomendePostcode_103"]) # Alleen voor buurten ingevuld.

0    Nederland                               
1    Aa en Hunze                             
2    Aa en Hunze                             
3    Aa en Hunze                             
4    Aa en Hunze                             
5    Aa en Hunze                             
6    Aa en Hunze                             
7    Aa en Hunze                             
8    Aa en Hunze                             
9    Aa en Hunze                             
Name: Gemeentenaam_1, dtype: object
0    Land      
1    Gemeente  
2    Wijk      
3    Buurt     
4    Buurt     
5    Wijk      
6    Buurt     
7    Buurt     
8    Wijk      
9    Buurt     
Name: SoortRegio_2, dtype: object
0    NL00      
1    GM1680    
2    WK168000  
3    BU16800000
4    BU16800009
5    WK168001  
6    BU16800100
7    BU16800109
8    WK168002  
9    BU16800200
Name: Codering_3, dtype: object


In [60]:
# Functie werkt maar automatisch queries samenvoegen is handiger.

#https://opendata.cbs.nl/ODataApi/odata/84583NED/TypedDataSet?$top=9999%20&amp;$filter=contains(Codering_3,%27WK%27)
htmlcode = {r" ":"%20", r'"':"%27", r"'":"%27"} # '%20' of 'amp;'' i.p.v. '&'' ?

def htmlvertaler(url_filter):
    htmlcode = {r" ":"%20", r'"':"%27", r"'":"%27"} # '%20' i.p.v. '&'' ?
    for h in htmlcode:
        while re.search(h, url_filter):
            url_filter = re.sub(h, htmlcode[h], url_filter)
    

    return url_filter
    
def get_top_observations(table_url, url_filter = "", skip=None):
    
    """Haal de tabel met metingen op. In tegenstelling tot Odata4 hoeft de tabel niet geformatteerd te worden.
    Odata3 beperkt echter hoeveel metingen je op kan halen. Kies uit de bovenste of onderste 9999 rijen.
    Als dat niet genoeg is moeten filters toegepast worden.
    Filteren om minder of specifiekere data op te vragen is mogelijk as volgt:
    url_filter = $filter=WijkenEnBuurten eq 'GM0363' and Measure eq 'T001036'
    Door de filteren op deze kolommen kun je een plaats (land, gemeente, wijk of buurt) kiezen.
    Of gebruik: $filter=Measure in ('M001607', 'M001636')
    Je hoeft niet beide kolommen te gebruiken voor het filter."""
    
    
    if skip:
        side = "$top=9999&$skip={}".format(skip)
    else:
        side = "$top=9999"
    
    if url_filter == "":
        target_url = table_url + "/TypedDataSet" + "?" + side 
    elif "$filter=" in url_filter:
        target_url = table_url + "/TypedDataSet" + "?" + url_filter + "&" + side # '%20' i.p.v. '&'' ?
    else:
        print("WAARSCHUWING! FILTER NIET GOED GEFORMATTEERD. VERGELIJK MET VOORBEELD OF GA NAAR ")
        print("\n https://www.odata.org/documentation/odata-version-3-0/url-conventions/")
        #return None
        return(table_url + url_filter)

    
    print(target_url)
    data = get_odata(target_url)
    #print(data.head)
    
    #if len(data.index) >= 9999:
    #    print("Waarschuwing: Verzoek om data ingekort door limiet van 9999 rijen.")
    #    print("Query '{}' is te groot voor één request".format(target_url))
    
    return(data)

In [61]:
def auto_query(url, fil):
    
    repeat = True
    nrow=0
    frames=[]
    
    while repeat:
        result = get_top_observations(url, url_filter=fil,skip=nrow)
        if len(result) != 9999:
            repeat = False
        nrow += 9999
        frames.append(result)
    
    return pd.concat(frames,axis=0)

In [62]:
def splits_cijfer(Identifier):
    if re.search("_\d+$", Identifier):
        cijfer = re.search("\d+$", Identifier).group()
    else:
        cijfer = "0"
    return cijfer
        

# Het slechte nieuws: De filters werken voor geen meter
# Het goede nieuws: Het ophalen van 9999 rijen garandeerd dat de query geldig is.
def get_observations(database, filtersoort=0, plaatsen = None, custom_filter="", features = None, beta=True):
    
    """
    Haal de tabel met metingen op. In tegenstelling tot Odata4 hoeft de tabel niet geformatteerd te worden.
    De code van de dataset is de combinatie van cijfers en letters achteraan de url:
    https://opendata.cbs.nl/ODataApi/odata/84583NED
    filtersoort geeft aan op welke kolom de plaatsen gefilterd worden.
    0 = alleen ingevoerde filter
    1 = naam gemeente
    2 = Gemeente/Wijk/Buurt
    3 = Code
    Je kan ook zelf een filter genereren dat achter de andere filters geplakt wordt. Zie 
    https://www.odata.org/documentation/odata-version-3-0/odata-version-3-0-core-protocol/
    voor meer uitleg. of ga naar 
    https://help.nintex.com/en-US/insight/OData/HE_CON_ODATAQueryCheatSheet.htm
    voor een beknopte cheatsheet
    """
    table_url = "https://opendata.cbs.nl/ODataApi/odata/{}".format(database) 
    # /TypedDataSet wordt er nog aangeplakt in get_top_observations()
    #print(table_url)
    
    if custom_filter != "":
            custom_filter = " & "+custom_filter
    
    if filtersoort == 1:
        filterkolom = "Gemeentenaam_1"
    elif filtersoort == 2:
        filterkolom = "SoortRegio_2"
    elif filtersoort == 3:
        filterkolom = "Codering_3"
    else:
        df = get_odata(table_url+"$filter="+custom_filter)
        
        return df
    
    frames = []
    if type(plaatsen) == list or type(plaatsen) == pd.Series:
        
        for plaats in plaatsen:
            url_filter = "$filter={} eq '".format(filterkolom)+plaats+"'"+custom_filter
            frames.append( auto_query(table_url,url_filter) )
            time.sleep(0.5)
            
        print("Download geslaagd!")
        observations = pd.concat(frames,axis=0)       
    
        if beta:
            observations[filterkolom] = observations[filterkolom].str.strip()
            frames = []
            for plaats in plaatsen:
                print(observations[filterkolom].head())
                frames.append( observations[observations[filterkolom] == plaats] )
            observations = pd.concat(frames,axis=0)
                
            
    
    else:
        observations = get_top_observations(table_url+custom_filter)
        # Resultaten die niet meer in 9999 rijen passen worden weggelaten! Maar dit voorkomt foutmeldingen.
        # Gebruik daarom een filter
    cols = pd.Series(observations.columns)
    Volgorde = cols.apply(lambda s: splits_cijfer(s))
    sorteer = pd.DataFrame({'Identifier':cols, 'Volgorde':Volgorde})
    sorteer['Volgorde'] = sorteer['Volgorde'].astype('int')
    sorteer = sorteer.sort_values('Volgorde').reset_index(drop=True)
    
    if features:
        return observations[features]
    else:
        return(observations[ list(sorteer['Identifier']) ])
    


Wijken ophalen gaat net met twee requests. Voor buurten zijn er dus meerdere nodig.

In [63]:
test_url1 = "https://opendata.cbs.nl/ODataApi/odata/84583NED/TypedDataSet?$top=9999%20&amp;$filter=contains(Codering_3,%27WK%27)"
test_url2 = "https://opendata.cbs.nl/ODataApi/odata/84583NED/TypedDataSet?$top=9999&$filter=contains(Codering_3,'WK')"
htmlvertaler(test_url2)


'https://opendata.cbs.nl/ODataApi/odata/84583NED/TypedDataSet?$top=9999&$filter=contains(Codering_3,%27WK%27)'

In [64]:
test_table = get_top_observations(lastyear_url, url_filter=htmlvertaler("$filter=Codering_3 eq 'WK195212'"))
#test_table = get_odata(lastyear_url+"/TypedDataSet"+"?$top=9999"+"&amp;"+"$filter=contains(Codering_3,'GM')")
#test_table = get_odata(lastyear_url+"/TypedDataSet"+"?$top=9999"+"%20&amp;"+"?$filter=contains(Codering_3,%27WK%27)")

print(test_table.shape)
test_table['Codering_3'].head(50)

https://opendata.cbs.nl/ODataApi/odata/84583NED/TypedDataSet?$filter=Codering_3%20eq%20%27WK195212%27&$top=9999
(9999, 108)


0     NL00      
1     GM1680    
2     WK168000  
3     BU16800000
4     BU16800009
5     WK168001  
6     BU16800100
7     BU16800109
8     WK168002  
9     BU16800200
10    BU16800209
11    WK168003  
12    BU16800300
13    BU16800309
14    WK168004  
15    BU16800400
16    BU16800409
17    WK168005  
18    BU16800500
19    BU16800509
20    WK168006  
21    BU16800600
22    BU16800609
23    WK168007  
24    BU16800700
25    BU16800709
26    WK168008  
27    BU16800800
28    BU16800809
29    WK168009  
30    BU16800900
31    BU16800909
32    WK168011  
33    BU16801100
34    BU16801109
35    WK168012  
36    BU16801200
37    BU16801209
38    WK168013  
39    BU16801300
40    BU16801309
41    WK168014  
42    BU16801400
43    BU16801401
44    BU16801409
45    WK168015  
46    BU16801500
47    BU16801501
48    BU16801509
49    WK168016  
Name: Codering_3, dtype: object

In [65]:
test_table = get_top_observations(lastyear_url, url_filter="$filter=contains(Codering_3,'WK')", skip = 9999)
#test_table = get_odata(lastyear_url+"/TypedDataSet"+"?$top=9999"+"&amp;"+"$filter=contains(Codering_3,'GM')")
print(test_table.shape)
test_table['Codering_3'].head()

https://opendata.cbs.nl/ODataApi/odata/84583NED/TypedDataSet?$filter=contains(Codering_3,'WK')&$top=9999&$skip=9999
(7115, 108)


0    BU19521301
1    BU19521302
2    BU19521303
3    WK195214  
4    BU19521401
Name: Codering_3, dtype: object

In [66]:
test = get_observations('84799NED',filtersoort=2, plaatsen=['Wijk'], beta=True)
test.tail(50)
#https://opendata.cbs.nl/ODataApi/odata/84583NED/TypedDataSet?$top=9999%20&amp;$filter=contains(SoortRegio_2,%27WK%27)

https://opendata.cbs.nl/ODataApi/odata/84799NED/TypedDataSet?$filter=SoortRegio_2 eq 'Wijk'&$top=9999
https://opendata.cbs.nl/ODataApi/odata/84799NED/TypedDataSet?$filter=SoortRegio_2 eq 'Wijk'&$top=9999&$skip=9999
Download geslaagd!
0        Land
1    Gemeente
2        Wijk
3       Buurt
4       Buurt
Name: SoortRegio_2, dtype: object


,WijkenEnBuurten,ID,Gemeentenaam_1,SoortRegio_2,Codering_3,IndelingswijzigingWijkenEnBuurten_4,AantalInwoners_5,Mannen_6,Vrouwen_7,k_0Tot15Jaar_8,...,AfstandTotKinderdagverblijf_94,AfstandTotSchool_95,ScholenBinnen3Km_96,OppervlakteTotaal_97,OppervlakteLand_98,OppervlakteWater_99,MeestVoorkomendePostcode_100,Dekkingspercentage_101,MateVanStedelijkheid_102,Omgevingsadressendichtheid_103
6976,WK063803,16975,Zoeterwoude,Wijk,WK063803,1,355,180,175,40,...,None,None,None,261,247,14,.,NaN,5.0,39.0
6978,WK063804,16977,Zoeterwoude,Wijk,WK063804,1,75,40,35,10,...,None,None,None,258,253,5,.,NaN,5.0,96.0
6980,WK063809,16979,Zoeterwoude,Wijk,WK063809,1,80,40,40,10,...,None,None,None,1008,976,31,.,NaN,5.0,76.0
6982,WK063810,16981,Zoeterwoude,Wijk,WK063810,1,2985,1500,1485,570,...,None,None,None,236,226,10,.,NaN,3.0,1123.0
6992,WK189201,16991,Zuidplas,Wijk,WK189201,1,4965,2480,2485,1115,...,None,None,None,843,839,4,.,NaN,5.0,495.0
7001,WK189202,17000,Zuidplas,Wijk,WK189202,1,8405,4190,4215,1590,...,None,None,None,2440,2034,406,.,NaN,4.0,729.0
7016,WK189203,17015,Zuidplas,Wijk,WK189203,1,8730,4390,4345,1490,...,None,None,None,1273,1213,60,.,NaN,3.0,1071.0
7029,WK189204,17028,Zuidplas,Wijk,WK189204,1,21795,10725,11070,3730,...,None,None,None,1849,1716,133,.,NaN,2.0,1534.0
7050,WK087900,17049,Zundert,Wijk,WK087900,1,8210,4115,4095,1110,...,None,None,None,1783,1777,6,.,NaN,4.0,854.0
7053,WK087901,17052,Zundert,Wijk,WK087901,1,2420,1280,1140,325,...,None,None,None,2532,2519,13,.,NaN,5.0,282.0


In [42]:
'   test   '.strip()

'test'

In [107]:
# Veel lege ruimte in sommige tekstkolommen, waaronder kolom 2.
print(test['SoortRegio_2'].unique())
regios = test['SoortRegio_2'].unique()
bestand = test[test['SoortRegio_2'] == regios[3]]
bestand.head(50)

['Land      ' 'Gemeente  ' 'Wijk      ' 'Buurt     ']


,WijkenEnBuurten,ID,Gemeentenaam_1,SoortRegio_2,Codering_3,IndelingswijzigingWijkenEnBuurten_4,AantalInwoners_5,Mannen_6,Vrouwen_7,k_0Tot15Jaar_8,...,AfstandTotKinderdagverblijf_97,AfstandTotSchool_98,ScholenBinnen3Km_99,OppervlakteTotaal_100,OppervlakteLand_101,OppervlakteWater_102,MeestVoorkomendePostcode_103,Dekkingspercentage_104,MateVanStedelijkheid_105,Omgevingsadressendichtheid_106
3,BU16800000,3,Aa en Hunze,Buurt,BU16800000,1,3450,1675,1775,555,...,0.5,0.8,2.6,252,252,0,9468,1.0,5.0,407.0
4,BU16800009,4,Aa en Hunze,Buurt,BU16800009,1,150,80,75,15,...,1.7,2.0,1.2,1226,1213,13,9468,1.0,5.0,59.0
6,BU16800100,6,Aa en Hunze,Buurt,BU16800100,1,1245,635,615,185,...,1.2,0.9,1.0,171,170,1,9463,1.0,5.0,144.0
7,BU16800109,7,Aa en Hunze,Buurt,BU16800109,1,175,90,85,10,...,2.0,2.1,1.3,1618,1611,6,9463,1.0,5.0,36.0
9,BU16800200,9,Aa en Hunze,Buurt,BU16800200,1,335,165,170,35,...,2.2,0.4,2.0,57,57,0,9467,1.0,5.0,126.0
10,BU16800209,10,Aa en Hunze,Buurt,BU16800209,1,130,85,50,15,...,2.3,0.9,1.9,759,757,1,9467,1.0,5.0,170.0
12,BU16800300,12,Aa en Hunze,Buurt,BU16800300,1,395,200,195,25,...,5.0,2.7,0.8,54,54,0,9466,1.0,5.0,53.0
13,BU16800309,13,Aa en Hunze,Buurt,BU16800309,1,15,5,10,0,...,4.2,2.0,0.8,860,851,8,9466,1.0,5.0,21.0
15,BU16800400,15,Aa en Hunze,Buurt,BU16800400,1,225,120,110,15,...,4.2,4.2,0.0,43,43,0,9465,1.0,5.0,33.0
16,BU16800409,16,Aa en Hunze,Buurt,BU16800409,1,30,20,15,5,...,3.6,3.6,0.4,1063,1061,1,9465,1.0,5.0,16.0


In [108]:
bestand.shape

(13594, 108)

In [109]:
test.to_csv("KerncijfersWijken2020.csv", sep=";")

## Filteren van query
### Werkt niet. Zoek uit wat er mis gaat, wijzig deze tekst en corrigeer de code
Het filteren van de data maakt het downloaden sneller.
Het filteren van 'Observations' data kan door code van dit format achter de url te plakken:

**?$filter=Codering_3 eq 'GM0363' and Measure eq 'T001036'**

**$filter=Measure in ('M001607', 'M001636')**

De code uit de kolom WijkenEnBuurten kun je vinden met 

**get_odata(table_url + "/WijkenEnBuurtenCodes")**

De 'Title' kolom van deze tabel bevat de namen van wijken, zodat je kan zoeken met str.find, <>.str.contains of Regex.
Zoals wel vaken met tektskolommen moet je dan vertrouwen op de volledigheid en consistentie.
Achteraf controleren of je wel de juiste weijken hebt s dus wel aangeraden.
De kolom WijkenenBuurten bevat zowel landen, gemeenten, wijken als buurten.
Aan het voorvoegsel van twee letters kun je zien met welke soort regio je te maken hebt.

Zie https://www.odata.org/documentation/odata-version-3-0/odata-version-3-0-core-protocol/ voor meer uitleg.
of ga naar https://help.nintex.com/en-US/insight/OData/HE_CON_ODATAQueryCheatSheet.htm voor een beknopte cheatsheet

Een dataset uit ODATA3 (uit 2019) kan gebruikt worden om postcodes aan buurten te koppelen als andere geografische data niet gevonden kan worden.
Houd rekening met de limiet van 9999 rijen per verzoek.

In [11]:
pd.Series(['AM','PM','FM']).str[-1]

0    M
1    M
2    M
dtype: object